In [7]:
import requests

In [8]:
def get_json_for_state(state):

    url = f'https://static01.nyt.com/elections-assets/2020/data/api/2020-11-03/state-page/{state}.json'
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        print(
            f"Something went wrong with state {state}. I'm outputing the raw response")
        return r

In [28]:
california_json = get_json_for_state('pennsylvania')['data']

In [30]:
pres_race = list(filter(lambda x: x["race_name"] == "President", california_json['races']))[0]

In [31]:
pres_race.keys()

dict_keys(['race_id', 'race_slug', 'url', 'state_page_url', 'ap_polls_page', 'edison_exit_polls_page', 'race_type', 'election_type', 'election_date', 'runoff', 'race_name', 'office', 'officeid', 'nyt_race_description', 'race_rating', 'seat', 'seat_name', 'state_id', 'state_slug', 'state_name', 'state_nyt_abbrev', 'state_shape', 'party_id', 'uncontested', 'report', 'result', 'result_source', 'gain', 'lost_seat', 'votes', 'electoral_votes', 'absentee_votes', 'absentee_counties', 'absentee_count_progress', 'absentee_outstanding', 'absentee_max_ballots', 'provisional_outstanding', 'provisional_count_progress', 'poll_display', 'poll_countdown_display', 'poll_waiting_display', 'poll_time', 'poll_time_short', 'precincts_reporting', 'precincts_total', 'reporting_display', 'reporting_value', 'eevp', 'tot_exp_vote', 'eevp_source', 'eevp_value', 'eevp_display', 'county_data_source', 'incumbent_party', 'no_forecast', 'last_updated', 'candidates', 'has_incumbent', 'leader_margin_value', 'leader_mar

In [34]:
pres_timeseries = pres_race["timeseries"]
pres_timeseries[0]

{'vote_shares': {'trumpd': 0, 'bidenj': 0},
 'votes': 0,
 'eevp': 0,
 'eevp_source': 'edison',
 'timestamp': '2020-11-04T09:25:23Z'}

In [75]:
def print_discrepancy(before_id):
    before_update = pres_timeseries[before_id]
    after_update = pres_timeseries[before_id+1]
    
    votes_djt_before = round(before_update["vote_shares"]["trumpd"] * before_update["votes"])
    votes_jrb_before = round(before_update["vote_shares"]["bidenj"] * before_update["votes"])
    votes_djt_after = round(after_update["vote_shares"]["trumpd"] * after_update["votes"])
    votes_jrb_after = round(after_update["vote_shares"]["bidenj"] * after_update["votes"])

    diff_djt = votes_djt_after - votes_djt_before
    diff_jrb = votes_jrb_after - votes_jrb_before
    diff_total = after_update["votes"] - before_update["votes"]
    
    
    print("DISCREPANCY DETECTED:\n")
    print("DJT VOTES BEFORE: {}".format(votes_djt_before))
    print("JRB VOTES BEFORE: {}".format(votes_jrb_before))
    print("TOTAL BEFORE: {}".format(before_update["votes"]))
    print("TIMESTAMP BEFORE: {}\n".format(before_update["timestamp"]))

    print("DJT VOTES AFTER: {}".format(votes_djt_after))
    print("JRB VOTES AFTER: {}".format(votes_jrb_after))
    print("TOTAL AFTER: {}".format(after_update["votes"]))
    print("TIMESTAMP AFTER: {}\n".format(after_update["timestamp"]))


    print("DJT DIFF: {}".format(diff_djt))
    print("JRB DIFF: {}".format(diff_jrb))
    print("TOTAL DIFF: {}".format(diff_total))
    print("===================================")

In [76]:
for i,update in enumerate(pres_timeseries):
    before_update = pres_timeseries[i]
    after_update = pres_timeseries[i+1]
    
    # Calculate vote counts
    votes_djt_before = round(before_update["vote_shares"]["trumpd"] * before_update["votes"])
    votes_jrb_before = round(before_update["vote_shares"]["bidenj"] * before_update["votes"])
    votes_djt_after = round(after_update["vote_shares"]["trumpd"] * after_update["votes"])
    votes_jrb_after = round(after_update["vote_shares"]["bidenj"] * after_update["votes"])
    
    if((votes_djt_after < votes_djt_before) or (votes_jrb_after < votes_jrb_before)):
        print_discrepancy(i)
    
    if(i==len(pres_timeseries)-2):
        break
    
    

DISCREPANCY DETECTED:

DJT VOTES BEFORE: 33
JRB VOTES BEFORE: 44
TOTAL BEFORE: 77
TIMESTAMP BEFORE: 2020-11-04T00:19:27Z

DJT VOTES AFTER: 0
JRB VOTES AFTER: 0
TOTAL AFTER: 0
TIMESTAMP AFTER: 2020-11-04T00:30:20Z

DJT DIFF: -33
JRB DIFF: -44
TOTAL DIFF: -77
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 4
JRB VOTES BEFORE: 7
TOTAL BEFORE: 11
TIMESTAMP BEFORE: 2020-11-04T00:42:45Z

DJT VOTES AFTER: 0
JRB VOTES AFTER: 0
TOTAL AFTER: 0
TIMESTAMP AFTER: 2020-11-04T00:59:15Z

DJT DIFF: -4
JRB DIFF: -7
TOTAL DIFF: -11
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 37039
JRB VOTES BEFORE: 180154
TOTAL BEFORE: 219166
TIMESTAMP BEFORE: 2020-11-04T01:17:19Z

DJT VOTES AFTER: 37055
JRB VOTES AFTER: 180011
TOTAL AFTER: 219258
TIMESTAMP AFTER: 2020-11-04T01:24:10Z

DJT DIFF: 16
JRB DIFF: -143
TOTAL DIFF: 92
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 47567
JRB VOTES BEFORE: 256553
TOTAL BEFORE: 306881
TIMESTAMP BEFORE: 2020-11-04T01:29:03Z

DJT VOTES AFTER: 177822
JRB VOTES AFTER: 256465
TOTAL AFTER: 436908
TIMES